In [1]:
import time
import rlapy as rla
import numpy as np
import scipy.linalg as la
from notebooks.least_squares.helpers import make_demo_helper


def time_alg(dims, spectrum, prop_range, alg, tol, seeds, lapack=False):
    n_rows, n_cols = dims
    lapack_times = np.zeros(seeds.size)
    rlapy_times = np.zeros(seeds.size)
    iter_lim = dims[1]
    rng = np.random.default_rng(seeds[0])
    dh = make_demo_helper(n_rows, n_cols, spectrum, prop_range, rng)
    A, b = dh.A, dh.b
    for i, seed in enumerate(seeds):
        if i > 0:
            rng = np.random.default_rng(seed)
            b = dh.resample_b(prop_range, rng)
        if lapack:
            tic = time.time()
            la.lstsq(A, b)
            lapack_times[i] = time.time() - tic
        tic = time.time()
        alg(A, b, tol, iter_lim, rng)
        rlapy_times[i] = time.time() - tic
    return lapack_times, rlapy_times

In [2]:
main_rng = np.random.default_rng(897389723094)
num_runs = 2
run_lapack = True
seeds = main_rng.integers(0, int(1e10), size=num_runs)
dims = (100000, 2000)
kappa = 1e5
prop_range = 0.95

sap = rla.SAP1(rla.srct_operator, sampling_factor=5)
print('Sketch and precondition configuration')
print('\tUse SRCT to sketch (discrete cosine transform)')
print('\tUse embedding dimension d = 5 * A.shape[1]\n')

# linearly decaying spectrum
lin_spec = np.linspace(kappa**0.5, kappa**-0.5, num=dims[1])
llapackt, lrandlat = time_alg(dims, lin_spec, prop_range,
                              sap, 1e-12, seeds, run_lapack)
print('Runtimes: A has linearly decaying spectrum')
print('\tLAPACK: ' + str(llapackt))
print('\trlapy : ' + str(lrandlat))
print()

# exponentially decaying spectrum
exp_spec = np.logspace(np.log10(kappa)/2, -np.log10(kappa)/2, num=dims[1])
elapackt, erandlat = time_alg(dims, exp_spec, prop_range,
                              sap, 1e-12, seeds, run_lapack)
print('Runtimes: A has exponentially decaying spectrum')
print('\tLAPACK: ' + str(elapackt))
print('\trlapy : ' + str(erandlat))

Sketch and precondition configuration
	Use SRCT to sketch (discrete cosine transform)
	Use embedding dimension d = 5 * A.shape[1]

Runtimes: A has linearly decaying spectrum
	LAPACK: [20.97871494 21.50359702]
	rlapy : [13.80186653 12.04853368]

Runtimes: A has exponentially decaying spectrum
	LAPACK: [19.69154763 20.62780356]
	rlapy : [12.01751542 12.64846778]


In [3]:
run_lapack = False  # LAPACK times are the same as in the last cell.
llapackt_old = llapackt.copy()
elapackt_old = elapackt.copy()

sap.sketch_op_gen = rla.sjlt_operator
print('Sketch and precondition configuration')
print('\tUse sparse Johnson-Lindenstrauss to sketch')
print('\tUse embedding dimension d = 5 * A.shape[1]\n')

llapackt, lrandlat = time_alg(dims, lin_spec, prop_range,
                              sap, 1e-12, seeds, run_lapack)
print('Runtimes: A has linearly decaying spectrum')
print('\tLAPACK: ' + str(llapackt_old))
print('\trlapy : ' + str(lrandlat))

print('\nRuntimes: A has exponentially decaying spectrum')
elapackt, erandlat = time_alg(dims, exp_spec, prop_range,
                              sap, 1e-12, seeds, run_lapack)
print('\tLAPACK: ' + str(elapackt_old))
print('\trlapy : ' + str(erandlat))

Sketch and precondition configuration
	Use sparse Johnson-Lindenstrauss to sketch
	Use embedding dimension d = 5 * A.shape[1]

Runtimes: A has linearly decaying spectrum
	LAPACK: [20.97871494 21.50359702]
	rlapy : [8.75145411 8.72003341]

Runtimes: A has exponentially decaying spectrum
	LAPACK: [19.69154763 20.62780356]
	rlapy : [9.45470905 8.85090303]
